# Multikey

We decided to use Multikey to represent Secp256k1 xOnly public keys in the DID document. This notebook walks through that process.

In [1]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey

mnemonic = "horse script comic poverty wink weekend cheese media apart barely major bag crop wheat private bicycle good nasty trouble fatal again whale permit three"
root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")

print("Mnemonic : ", mnemonic)

Mnemonic :  horse script comic poverty wink weekend cheese media apart barely major bag crop wheat private bicycle good nasty trouble fatal again whale permit three


# Generate a XOnly Secp256k1 public key

Represented in bytes

In [2]:
didkey_purpose = "11"
initial_vm_privkey = root_hdpriv.get_private_key(didkey_purpose, address_num=4)
xonly_publickey = initial_vm_privkey.point.xonly()


xonly_publickey

b'\x97\x9a\xc3\x83\x05\xa6\xd6\x07J\xc7\xfd.bp&\xf1\xa6\xaa\xfd\x0e@\xe3\xdc\xbc\x16_W\x82\x94\xb8:S'

# Convert Bytes to Multikey

## Must select two byte prefix identifying the Multikey as a Secp256k1 XOnly public key

For example from the controller document spec:

**ECDSA 256-bit public key**: The Multikey encoding of a P-256 public key MUST start with the two-byte prefix 0x8024 (the varint expression of 0x1200) followed by the 33-byte compressed public key data. The resulting 35-byte value MUST then be encoded using the base-58-btc alphabet, according to Section 2.4 Multibase, and then prepended with the base-58-btc Multibase header (z).

They select the two-byte prefix as 0x8024 (the varint expression of 0x1200).

In [3]:
from multiformats import varint

In [4]:
varint.encode(0x1200).hex()

'8024'

## Proposal: XOnly Secp256k1 2-byte prefix is 0xe14a (the varint expression of 0x2561)

Note: We can debate this. I don't particularly care. I think we just need to select one.

In [15]:
secp_xonly_prefix = varint.encode(0x2561)

In [16]:
secp_xonly_prefix.hex()

'e14a'

In [17]:
varint.decode(secp_xonly_prefix) == 0x2561

True

In [18]:
multikey_bytes = secp_xonly_prefix + xonly_publickey

In [19]:
multikey_bytes

b'\xe1J\x97\x9a\xc3\x83\x05\xa6\xd6\x07J\xc7\xfd.bp&\xf1\xa6\xaa\xfd\x0e@\xe3\xdc\xbc\x16_W\x82\x94\xb8:S'

## Must select encoding: Propose using base58btc.

Note: we might want to use bech32 here. But until we can get this in the multibase specification this will not be possible

In [20]:
from multiformats import multibase

In [21]:
multikey_value = multibase.encode(multikey_bytes, "base58btc")

print("Multikey value : ", multikey_value)

Multikey value :  z66Pw6AyJsXnpc6bNQ93FwjYuatdnACMyQLLqqxDea394MxN


In [24]:
multikey_value[2:]

'6Pw6AyJsXnpc6bNQ93FwjYuatdnACMyQLLqqxDea394MxN'

## Spec Text

**Secp256k1 XOnly public key**: The Multikey encoding of a Secp256k1 XOnly public key MUST start with the two-byte prefix 0xe14a (the varint expression of 0x2561) followed by the 32-byte XOnly public key data. The resulting 34-byte value MUST then be encoded using the base-58-btc alphabet, according to Section 2.4 Multibase in the Controller Document specification, and then prepended with the base-58-btc Multibase header (z).


## Note: Must also propose a two-byte value to identify the secret key

Propse: 0x130e

**Secp256k1 secret key** The Multikey encoding of a Secp256k1 secret key MUST start with the two-byte prefix 0x8626 (the varint expression of 0x1306) followed by the 32-byte secret key data. The resulting 34-byte value MUST then be encoded using the base-58-btc alphabet, according to Section 2.4 Multibase, and then prepended with the base-58-btc Multibase header (z).

In [11]:
varint.encode(0x130e).hex()

'8e26'

## Construct Verification Method

In [41]:

verificationMethod = {}
verificationMethod["id"] = '#initialKey'
verificationMethod["type"] = "Multikey"
verificationMethod["publicKeyMultibase"] = multikey_value


In [42]:
verificationMethod

{'id': '#initialKey',
 'type': 'Multikey',
 'publicKeyMultibase': 'z66Pw6AyJsXnpc6bNQ93FwjYuatdnACMyQLLqqxDea394MxN'}